In [1]:
import pandas as pd
import numpy as np

In [2]:
from surprise import Dataset
from surprise import Reader

In [3]:
from UBCFBase import UBCFBase
from FS_Weight import FS_Weight
from surprise import KNNBasic
from FS_Threshold import FS_Threshold
from FR_JIZHI import FR_JIZHI
from FR_Weight import FR_Weight
from GFusion import GFusion

In [4]:
reader = Reader(line_format='user item rating', sep=':')

In [5]:
data = Dataset.load_from_file('new_usr_ratings.txt', reader=reader)

In [6]:
data.split(n_folds=3)

In [7]:
trainsets = []
testsets = []
for trainset, testset in data.folds():
    trainsets.append(trainset)
    testsets.append(testset)

In [8]:
first_trainset = trainsets[0]
first_testset = testsets[0]

In [9]:
first_trainset.n_users

971

In [10]:
algo_base = UBCFBase(alpha=0.5)

In [11]:
algo_base.train(first_trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the behavior similarity matrix...
Done computing behavior similarity matrix


In [12]:
print first_trainset.to_raw_uid(967)
print first_trainset.to_raw_uid(965)
print first_trainset.to_raw_uid(961)

152533709
only6
141561635


In [13]:
print first_trainset.to_raw_uid(345)
print first_trainset.to_raw_uid(34)
print first_trainset.to_raw_uid(431)

doumo.521
drtsb
123404248


In [14]:
user_comment_raw = algo_base.trainset_assist_rating_df.groupby(['uid', 'comment_type']).size().unstack().fillna(0)

In [15]:
user_comment_raw['sum'] = user_comment_raw[0] + user_comment_raw[1]

In [16]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [21]:
def plot_user_hot_hist(user, ax):
    temp=algo_base.trainset_assist_rating_df[algo_base.trainset_assist_rating_df.uid==user]
    temp = pd.merge(temp, algo_base.IHot, left_on='iid', right_index=True)
    temp['hot'].plot(ax=ax, kind='hist')

In [22]:
import matplotlib.pyplot as plt

In [39]:
fig, axes = plt.subplots(1, 2)

In [63]:
algo_base.URDI[23]

0.45307677654713241

In [41]:
user_date_raw = algo_base.trainset_assist_rating_df.groupby(['uid', 'date']).size()

In [56]:
user_date_raw[2].plot()

In [62]:
user_date_raw[23]

date
1800-01-01      1
2011-05-10      1
2011-05-12      3
2011-05-21    193
2011-05-22      1
2011-05-24      1
2011-06-11      4
2011-06-12      1
2011-07-24      2
2011-08-25      2
2011-10-04      2
2011-10-19      1
2011-11-09      1
2011-12-07      1
2011-12-08      1
2011-12-10      1
2011-12-12      2
2011-12-14      1
2011-12-25      2
2012-01-01     32
2012-01-20      2
2012-01-28      1
2012-02-13      2
2012-03-10      2
2012-03-13      1
2012-04-02      1
2012-04-10      2
2012-04-22      1
2012-04-24      1
2012-05-18      1
             ... 
2015-09-06      1
2015-09-07      1
2015-09-18      2
2015-09-19      1
2015-09-23      1
2016-01-08      1
2016-01-25      1
2016-02-13      1
2016-02-14      1
2016-06-14      1
2016-07-15      1
2016-07-23      1
2016-07-24      1
2016-08-31      1
2016-09-02      4
2016-09-11      1
2016-09-19      1
2016-09-29      1
2016-12-22      1
2016-12-28      2
2017-01-02      5
2017-01-08      1
2017-01-15      5
2017-01-22      2
2017-

In [65]:
algo_base.sim

array([[ 1.        ,  0.09608091,  0.08069164, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.09608091,  1.        ,  0.32258065, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.08069164,  0.32258065,  1.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  1.        ]])

In [66]:
algo_base.behavior_sim

array([[ 0.        ,  0.18400873,  0.17026726, ...,  0.28233277,
         0.27624027,  0.27283152],
       [ 0.18400873,  0.        ,  0.01942684, ...,  0.14210911,
         0.09224855,  0.10784282],
       [ 0.17026726,  0.01942684,  0.        , ...,  0.13889151,
         0.10758046,  0.11091543],
       ..., 
       [ 0.28233277,  0.14210911,  0.13889151, ...,  0.        ,
         0.12799598,  0.05084809],
       [ 0.27624027,  0.09224855,  0.10758046, ...,  0.12799598,
         0.        ,  0.07714789],
       [ 0.27283152,  0.10784282,  0.11091543, ...,  0.05084809,
         0.07714789,  0.        ]])

In [68]:
fs_weight = FS_Weight()

In [69]:
fs_weight.train(first_trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the behavior similarity matrix...
Done computing behavior similarity matrix


In [71]:
fs_weight.fusion_sim

array([[ 0.5       ,  0.14004482,  0.12547945, ...,  0.14116638,
         0.13812014,  0.13641576],
       [ 0.14004482,  0.5       ,  0.17100374, ...,  0.07105455,
         0.04612427,  0.05392141],
       [ 0.12547945,  0.17100374,  0.5       , ...,  0.06944576,
         0.05379023,  0.05545771],
       ..., 
       [ 0.14116638,  0.07105455,  0.06944576, ...,  0.5       ,
         0.06399799,  0.02542405],
       [ 0.13812014,  0.04612427,  0.05379023, ...,  0.06399799,
         0.5       ,  0.03857394],
       [ 0.13641576,  0.05392141,  0.05545771, ...,  0.02542405,
         0.03857394,  0.5       ]])

In [72]:
fs_weight_result = pd.DataFrame.from_csv('FS_Weight_04-13-03-39')

In [97]:
fs_weight_result

,MAE,RMSE,alpha,beta,params,scores
0,1.318974,1.674446,0.0,0.0,"{'alpha': 0.0, 'beta': 0.0}","{'MAE': 1.3189738975087979, 'RMSE': 1.67444554..."
1,1.303790,1.661751,0.0,0.2,"{'alpha': 0.0, 'beta': 0.20000000000000001}","{'MAE': 1.3037898358829436, 'RMSE': 1.66175138..."
2,1.282637,1.643834,0.0,0.4,"{'alpha': 0.0, 'beta': 0.40000000000000002}","{'MAE': 1.2826371711205842, 'RMSE': 1.64383398..."
3,1.247780,1.614840,0.0,0.6,"{'alpha': 0.0, 'beta': 0.60000000000000009}","{'MAE': 1.2477799531144929, 'RMSE': 1.61484032..."
4,1.181330,1.560049,0.0,0.8,"{'alpha': 0.0, 'beta': 0.80000000000000004}","{'MAE': 1.1813296842700287, 'RMSE': 1.56004931..."
5,1.040216,1.428452,0.0,1.0,"{'alpha': 0.0, 'beta': 1.0}","{'MAE': 1.0402163864737168, 'RMSE': 1.42845206..."
6,1.319184,1.674697,0.2,0.0,"{'alpha': 0.20000000000000001, 'beta': 0.0}","{'MAE': 1.3191838646372396, 'RMSE': 1.67469711..."
7,1.303988,1.661989,0.2,0.2,"{'alpha': 0.20000000000000001, 'beta': 0.20000...","{'MAE': 1.3039877428608242, 'RMSE': 1.66198922..."
8,1.282803,1.644018,0.2,0.4,"{'alpha': 0.20000000000000001, 'beta': 0.40000...","{'MAE': 1.2828033183645555, 'RMSE': 1.64401760..."
9,1.247915,1.614983,0.2,0.6,"{'alpha': 0.20000000000000001, 'beta': 0.60000...","{'MAE': 1.2479154141307716, 'RMSE': 1.61498308..."


In [81]:
n_users = algo_base.trainset.n_users

In [82]:
n_users

971

In [84]:
placeholder = np.empty((n_users, n_users), dtype=bool)

user_rating_items = {}

for u in range(n_users):
    user_rating_items[u] = [item for item, rating in algo_base.trainset.ur[u]]

In [88]:
for i in range(n_users):
    for j in range(n_users):
        common_rate_count = len(np.intersect1d(user_rating_items[i], user_rating_items[j]))
        if common_rate_count > 10:
            placeholder[i, j] = True
        else:
            placeholder[i, j] = False

In [89]:
placeholder

array([[ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       ..., 
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]], dtype=bool)

In [94]:
np.where(placeholder, algo_base.sim, algo_base.behavior_sim)

array([[ 1.        ,  0.09608091,  0.08069164, ...,  0.28233277,
         0.27624027,  0.27283152],
       [ 0.09608091,  1.        ,  0.32258065, ...,  0.14210911,
         0.09224855,  0.10784282],
       [ 0.08069164,  0.32258065,  1.        , ...,  0.13889151,
         0.10758046,  0.11091543],
       ..., 
       [ 0.28233277,  0.14210911,  0.13889151, ...,  0.        ,
         0.12799598,  0.05084809],
       [ 0.27624027,  0.09224855,  0.10758046, ...,  0.12799598,
         0.        ,  0.07714789],
       [ 0.27283152,  0.10784282,  0.11091543, ...,  0.05084809,
         0.07714789,  0.        ]])

In [96]:
algo_base.behavior_sim

array([[ 0.        ,  0.18400873,  0.17026726, ...,  0.28233277,
         0.27624027,  0.27283152],
       [ 0.18400873,  0.        ,  0.01942684, ...,  0.14210911,
         0.09224855,  0.10784282],
       [ 0.17026726,  0.01942684,  0.        , ...,  0.13889151,
         0.10758046,  0.11091543],
       ..., 
       [ 0.28233277,  0.14210911,  0.13889151, ...,  0.        ,
         0.12799598,  0.05084809],
       [ 0.27624027,  0.09224855,  0.10758046, ...,  0.12799598,
         0.        ,  0.07714789],
       [ 0.27283152,  0.10784282,  0.11091543, ...,  0.05084809,
         0.07714789,  0.        ]])

In [98]:
_file = file('new_usr_ratings.txt')

In [99]:
a = _file.readlines()

In [101]:
import random

In [102]:
random.shuffle(a)

In [104]:
len(a)

587596

In [105]:
length = len(a) * 2 / 3

In [106]:
length

391730

In [108]:
len(a[0:length])

391730

In [109]:
a[0:length]

['jadycheung:3541415:4\n',
 'FUNA:10605978:0\n',
 'snakewong:1292722:5\n',
 'pvcflooring:6146955:5\n',
 'jadycheung:4271894:5\n',
 'justseverin:5681006:0\n',
 'chenolivia:11529526:3\n',
 '73399489:6844900:1\n',
 'lightwing86:2027933:4\n',
 'joypig:1293731:0\n',
 'HeresyBreak:11527487:4\n',
 'sophie1900:26351867:5\n',
 'peach_thy:2124724:3\n',
 '33620259:1291832:4\n',
 'Lai-_-:22759752:3\n',
 'jadycheung:2043465:3\n',
 'fxee:2080133:0\n',
 'momo13-:2148257:4\n',
 '54588016:26265170:0\n',
 'Ozu:6952149:5\n',
 'ohjoo:6710798:3\n',
 '57634132:24703986:4\n',
 'zeus93:3600972:0\n',
 'fxee:25858785:1\n',
 'TERRYBEAR:2994961:0\n',
 'luzhiyu:3041281:3\n',
 '45765669:1410556:3\n',
 'shinylinshan:1937621:5\n',
 '70263096:20513355:5\n',
 'kn0537:6537486:3\n',
 'bdhztian:24522982:0\n',
 'wyxwyx1993:24751756:4\n',
 'lephemera:2336785:3\n',
 'jinzeyu007:1417555:3\n',
 '46643232:1295686:4\n',
 '1719006:10733231:2\n',
 'redkop:1294202:4\n',
 'yinxiang:1986080:4\n',
 '46376950:5985961:5\n',
 'hana.x:147

In [111]:
train_file = open('new_usr_ratings.train', 'w')

In [112]:
train_file.writelines(a[0:length])

In [113]:
train_file.close()

In [114]:
c = [0, 1, 3, 4]

In [116]:
c[0:2]

[0, 1]

In [118]:
c[2:]

[3, 4]

In [122]:
test_file = open('new_usr_ratings.test', 'w')

In [123]:
test_file.writelines(a[length:])

In [124]:
test_file.close()

In [125]:
train_file = 'new_usr_ratings.train'
test_file = 'new_usr_ratings.test'

In [126]:
data = Dataset.load_from_folds([(train_file, test_file)], reader)

In [129]:
algo = KNNBasic()
from surprise import dump

In [130]:
for trainset, testset in data.folds():
    algo.train(trainset)
    predictions = algo.test(testset)
    dump('./dump_knn', predictions, trainset, algo)

Computing the msd similarity matrix...
Done computing similarity matrix.
('The dump has been saved as file', './dump_knn')


In [131]:
from surprise.accuracy import rmse

In [132]:
import pickle

In [133]:
dump_svk = pickle.load(open('./dump_knn', 'rb'))

In [134]:
predictions = dump_svk['predictions']

In [135]:
rmse(predictions)

RMSE: 1.4266


1.4265879991339414